<a href="https://colab.research.google.com/github/Stefano-Previti/Ancient_to_Modern_Italian_Automatic_Translation/blob/main/Ancient_to_Modern_Italian_AT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0:IMPORTINGN LIBRARIES

In [11]:
!pip install -U gdown
!pip install -q transformers sentencepiece torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 887.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.5 MB/s eta 0:00:00


In [12]:
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, pipeline
from google.colab import drive

# 1: LOAD DATA

In [3]:
!gdown --fuzzy "https://drive.google.com/file/d/1SylTNYz4Ipv6GX75IcGEMTKa-nn1ke32/view?usp=sharing" -O dataset

Downloading...
From: https://drive.google.com/uc?id=1SylTNYz4Ipv6GX75IcGEMTKa-nn1ke32
To: /content/dataset
100% 14.6k/14.6k [00:00<00:00, 40.4MB/s]


In [4]:
# Extract the data.
df = pd.read_csv('/content/dataset')
df.head()

,Author,Date,Region,Sentence
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...


In [5]:

# Rename the sentence column to 'source'.
df = df.rename(columns={"Sentence": "source"})

# Select the 'source' column.
dataset = df[["source"]].dropna().reset_index(drop=True)

# Export to JSON Lines format.
dataset.to_json(
    "input.jsonl",
    orient="records",
    lines=True,
    force_ascii=False
)

print(f" Created input.jsonl with {len(dataset)} records.")


 Created input.jsonl with 97 records.


In [6]:
!head -n 100 input.jsonl


{"source":"quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno"}
{"source":"crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi."}
{"source":"Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere."}
{"source":"Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati."}
{"source":"Officio di questa arte pare che sia dicere appostatamente per fare credere, fine è far credere per lo dire."}
{"source":"Ecco e larghi ventipiovoli caggiono delle risolute nebbie; e potresti credere che tutto il cielo cadesse nel mare"}
{"source":"Però che or chi spererebbe quello che eziandio questi che non vogliono ancora credere in Cristo, già veggiono con noi, e perché nol possono negare, stridono colli denti."}
{"source":"I vendimenti de' morti et le presure

# 2: TRANSFORMER BASED MACHINE TRANSLATION

## 2.1:mBART-50

mBART-50 is a multilingual Sequence-to-Sequence model pre-trained using the "Multilingual Denoising Pretraining" objective.

We tried to use a larger hypothesis space with 8 beams , a lenght-penalty, a repetition penalty and we disallowed any repeatition of 3-gram in the output .

Despite this, the model has some problem in the final output translating improperly some words in english instead of modern italian.


In [7]:
drive.mount('/content/drive')

# Device.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the pre-trained mBART-50 model and tokenizer.
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

# Configuration of the tokenizer for Italian-to-Italian generation
tokenizer.src_lang = "it_IT"
forced_bos_token_id = tokenizer.lang_code_to_id["it_IT"]

# Helper function to transform archaic Italian into modern Italian
def translate_ancient_to_modern(text: str) -> str:
    # Tokenize the input text
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    # Generate the modern Italian output
    generated_tokens = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=8,                              # use beam search with 8 beams for broader hypothesis exploration
    max_length=256,
    no_repeat_ngram_size=3,                   # prevent the model from repeating any 3-gram in the output
    length_penalty=1.2,                       # encourag longer and more complete outputs (penalize too-short sequences)
    repetition_penalty=2.0,                   # heavily penalize repetition of the same tokens
    use_cache=True,
)
    # Decode the generated tokens back into a string
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

# Apply the translation to each source sentence
df = pd.read_json("input.jsonl", lines=True)
df["translation"] = df["source"].apply(translate_ancient_to_modern)

# Export to a new JSONL file.
output_path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50.jsonl'
df.to_json(
    output_path,
    orient='records',
    lines=True,
    force_ascii=False
)


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [10]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-mBART-50.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print("-" * 40)

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: La guerra ha fatto bene, perché etc. E dall'altra parte Aiaces era un bravo guerriero, e brava per le armi, ma non pieno di grande senno.
----------------------------------------
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: crudele, e per tutte le colpe vendetta, come dice la legge, e da cavaliere perdon che pecchi.
----------------------------------------
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Non c'è altra forza d' animosità che è stato venerato il Ponzio dell'Umiliare, un romano cavaliere.
----------------------------------------
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non rite

## 2.2: M2M100

M2M100 is a multilingual encoder-decoder (seq-to-seq) model trained for Many-to-Many multilingual translation.

We set the same parameters as before for the generation phase.

Here we didn't suffer about the swithcing in other language in the output.

In [20]:
# Model and tokenizer,
MODEL = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(MODEL)
model     = M2M100ForConditionalGeneration.from_pretrained(MODEL).to(device)

# Italian as input and as output.
SRC_LANG = "it"
TGT_LANG = "it"
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG
forced_bos = tokenizer.lang_code_to_id[TGT_LANG]

# Translation.
def translation(text: str) -> str:
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(device)
    # Generation.
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos,
        num_beams=8,
        max_length=256,
        no_repeat_ngram_size=3,
        length_penalty=1.2,
        repetition_penalty=2.0,
        early_stopping=True
    )
    # Decode to string
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Read file.
df = pd.read_json("input.jsonl", lines=True)

# Translate and saving.
df["translation"] = df["source"].apply(translation)
output_path = "/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100.jsonl"
df.to_json(
    output_path,
    orient="records",
    lines=True,
    force_ascii=False
)



In [21]:
path = '/content/drive/MyDrive/Previti/Previti-hw2_transl-M2M100.jsonl'
df_out = pd.read_json(path, lines=True)

for idx, row in df_out.iterrows():
    print(f"SOURCE     : {row['source']}")
    print(f"TRANSLATION: {row['translation']}")
    print("-" * 40)

SOURCE     : quella guerra ben fatta l' opera perché etc. Et dall' altra parte Aiaces era uno cavaliere franco e prode all' arme, di gran guisa, ma non era pieno di grande senno
TRANSLATION: E dall'altra parte Aiaces era un cavaliere franco e prodo alle armi, di grande fortuna, ma non era pieno di grande seno.
----------------------------------------
SOURCE     : crudele, e di tutte le colpe pigli vendetta, come dice la legge, ed a neuno cavaliere perdoni che pecchi.
TRANSLATION: crudele, e di tutti i peccati piglia vendetta, come dice la legge, e a nove cavalieri perdono che peccano.
----------------------------------------
SOURCE     : Non d' altra forza d' animo fue ornato Ponzio Aufidiano, romano cavaliere.
TRANSLATION: Nessuna altra forza d'anima fu ornato Ponzio Aufidiano, cavaliere romano.
----------------------------------------
SOURCE     : Se questo piace a tutti e se 'l tempo hae bisogno d'avere Pompeio per cavaliere e non per compagno, non riterrò più i fati.
TRANSLATION: S